In [0]:
# Load the data from DBFS
df_gold = spark.read.parquet("dbfs:/Users/simhad76@students.rowan.edu/silver_layer_data")

# Display the first few rows of the DataFrame
print("Sample data from Gold Layer:")
display(df_gold.limit(25))

# Get the number of rows and columns
num_rows = df_gold.count()
num_columns = len(df_gold.columns)

# Print the number of rows and columns
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

# Print the column names
print("\nColumns:")
for column in df_gold.columns:
    print(column)

# Continue with further analysis or transformations...


Sample data from Gold Layer:


id,popularity,vote_count,title,release_date,vote_average,genre,original_language,Performance_Analysis
49948,14.509,1264.0,Fantasia 2000,2000-01-01,6.966,"Animation, Family, Music",en,Hit
41245,11.201,255.0,Peppermint Candy,2000-01-01,7.5,Drama,ko,Hit
46462,6.597,7.0,Nautilus,2000-01-01,3,"Action, Science Fiction",en,Hit
477164,5.274,0.0,A Profile of In Which We Serve,2000-01-01,0,Documentary,en,Hit
55178,4.95,23.0,D-Day,2000-01-01,5.7,"TV Movie, Crime, Drama",da,Hit
20152,4.865,29.0,Angels of the Universe,2000-01-01,6.9,Drama,is,Hit
196139,4.483,8.0,Helter Skelter,2000-01-01,2.6,Horror,en,Hit
56721,4.369,11.0,The Mummy Theme Park,2000-01-01,4.5,Horror,en,Hit
218188,4.038,1.0,Table One,2000-01-01,4,Comedy,en,Hit
625522,3.921,2.0,Erotic Day Dream,2000-01-01,5,Science Fiction,en,Hit


Number of rows: 104905
Number of columns: 9

Columns:
id
popularity
vote_count
title
release_date
vote_average
genre
original_language
Performance_Analysis


In [0]:
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, countDistinct, avg, when, count, year,  udf, month, to_date, avg, round, size
from pyspark.sql.functions import coalesce, max, min, desc, sum,first, last,expr,size,split,when, explode,lit,sum as _sum 
import random
from datetime import datetime
from pyspark.sql.types import IntegerType, StringType



In [0]:
# To analyze the performance of movies by genre and determine which genre has the highest average performance level while also showing the total movie count for each genre, you can follow these steps:

# Check for null values in the genre column
null_genre_count = df_gold.filter(col("genre").isNull()).count()
print(f"Number of null rows in genre column: {null_genre_count}")
# Count distinct genres
distinct_genres = df_gold.select("genre").distinct().count()
print(f"Number of distinct genres: {distinct_genres}")


# Assuming genres are stored as a string (e.g., "Action, Comedy"), we can split them
# Split the genre column by comma and explode it to create a new row for each genre
df_genres = df_gold.withColumn("genre", explode(split(col("genre"), ", ")))

# Group by genre and calculate average vote_average and total movie count
performance_by_genre = df_genres.groupBy("genre") \
    .agg(
        avg("vote_average").alias("average_performance"),
        count("*").alias("total_movie_count")
    ) \
    .orderBy(desc("average_performance"))

# Get the genre with the maximum average performance level
top_performance_genre = performance_by_genre.first()  # Get the top result

# Show results for each genre with its average performance and total movie count
print("\nPerformance analysis by genre:")
performance_by_genre.show(truncate=False)

# Show results for the genre with maximum performance level
print("\nGenre with maximum average performance level:")
if top_performance_genre:
    print(f"Genre: {top_performance_genre['genre']}, Average Performance Level: {top_performance_genre['average_performance']:.2f}, Total Movie Count: {top_performance_genre['total_movie_count']}")
else:
    print("No data available.")


Number of null rows in genre column: 0
Number of distinct genres: 4225

Performance analysis by genre:
+---------------+-------------------+-----------------+
|genre          |average_performance|total_movie_count|
+---------------+-------------------+-----------------+
|Adventure      |4.884286838340484  |2796             |
|Mystery        |4.549206854345166  |2451             |
|Family         |4.488424219536759  |3972             |
|Thriller       |4.467644222415288  |6906             |
|War            |4.41953616636528   |1106             |
|Romance        |4.411732318147498  |7946             |
|Fantasy        |4.380755629383535  |2709             |
|Crime          |4.349897260273967  |4380             |
|Action         |4.3128148035795535 |6593             |
|Science Fiction|4.250006664198443  |2701             |
|TV Movie       |4.242115463447964  |4391             |
|Western        |4.2080110294117645 |272              |
|Comedy         |4.056372274894214  |18669            |
|

In [0]:


def top_performing_languages_with_genre(df):
    """
    Identify top performing languages based on Blockbuster and Superhit films,
    and classify them as multi-genre, single-genre, or unknown genre.
    """
    # Filter for relevant performance analysis
    filtered_df = df.filter(
        (col("Performance_Analysis") == "Culthit") | 
        (col("Performance_Analysis") == "Superhit/Blockbuster")
    )

    # Determine if a film is multi-genre, single-genre, or unknown genre
    filtered_df = filtered_df.withColumn(
        "genre_count",
        size(split(col("genre"), ","))
    ).withColumn(
        "genre_type",
        when(col("genre").startswith("Unknown"), "unknown_genre")
        .when(col("genre_count") > 1, "multi_genre")
        .otherwise("single_genre")
    )

    # Group by language and genre type
    top_languages = filtered_df.groupBy("original_language", "genre_type") \
        .agg(
            count("*").alias("top_film_count"),
            round(avg("popularity"), 3).alias("avg_popularity")  # Round average popularity to 3 decimal places
        ).orderBy(col("top_film_count").desc(), col("avg_popularity").desc())
    
    return top_languages

# Get Top Performing Languages with Genre Type
top_languages = top_performing_languages_with_genre(df_gold)

print("Top Performing Languages (Culthit & Superhit Films):")
top_languages.show(truncate=False)

# Additional analysis for genre counts if needed
df_genre_counts = df_gold.withColumn(
    "genre_count",
    size(split(col("genre"), ","))
).groupBy(
    when(col("genre").isNull() | col("genre").startswith("Unknown"), "unknown_genre")
    .when(col("genre_count") > 1, "multi_genre")
    .otherwise("single_genre").alias("genre_type")
).agg(
    count("*").alias("film_count")
)

# Show the results for genre counts
print("Number of multi-genre, single-genre, and unknown genre films:")
df_genre_counts.show(truncate=False)


Top Performing Languages (Culthit & Superhit Films):
+-----------------+-------------+--------------+--------------+
|original_language|genre_type   |top_film_count|avg_popularity|
+-----------------+-------------+--------------+--------------+
|en               |multi_genre  |212           |84.657        |
|en               |single_genre |16            |72.527        |
|ja               |multi_genre  |9             |68.518        |
|es               |multi_genre  |6             |59.976        |
|ko               |multi_genre  |3             |61.515        |
|fr               |multi_genre  |3             |61.313        |
|it               |multi_genre  |2             |173.739       |
|es               |single_genre |2             |53.218        |
|nl               |single_genre |1             |314.297       |
|it               |single_genre |1             |79.477        |
|ja               |unknown_genre|1             |68.146        |
|lt               |single_genre |1             |66.

In [0]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, count, avg, size, split, when, round

def top_performing_languages_with_genre(df: DataFrame):
    """
    Identify top performing languages based on Blockbuster and Superhit films,
    and classify them as multi-genre, single-genre, or unknown genre.
    Also, calculate the average popularity by genre type.
    """
    # Filter for relevant performance analysis
    filtered_df = df.filter(
        (col("Performance_Analysis") == "Culthit") | 
        (col("Performance_Analysis") == "Superhit/Blockbuster")
    )

    # Determine if a film is multi-genre, single-genre, or unknown genre
    filtered_df = filtered_df.withColumn(
        "genre_count",
        size(split(col("genre"), ","))
    ).withColumn(
        "genre_type",
        when(col("genre").startswith("Unknown"), "unknown_genre")
        .when(col("genre_count") > 1, "multi_genre")
        .otherwise("single_genre")
    )

    # Group by genre_type and calculate metrics
    top_genre_types = filtered_df.groupBy("genre_type") \
        .agg(
            count("*").alias("top_film_count"),
            round(avg("popularity"), 3).alias("avg_popularity")  # Round average popularity to 3 decimal places
        ).orderBy(col("top_film_count").desc(), col("avg_popularity").desc())
    
    return top_genre_types

# Get Top Performing Genres (Culthit & Superhit Films)
top_genre_types = top_performing_languages_with_genre(df_gold)

print("Top Performing Genres (Culthit & Superhit Films):")
top_genre_types.show(truncate=False)

# Additional analysis for genre counts if needed
df_genre_counts = df_gold.withColumn(
    "genre_count",
    size(split(col("genre"), ","))
).withColumn(
    "genre_type",
    when(col("genre").isNull() | col("genre").startswith("Unknown"), "unknown_genre")
    .when(col("genre_count") > 1, "multi_genre")
    .otherwise("single_genre")
).groupBy(
    "genre_type"
).agg(
    count("*").alias("film_count"),
    round(avg("popularity"), 3).alias("avg_popularity")  # Calculate average popularity for genre types
)

# Show the results for genre counts and average popularity
print("Number of films by genre type and their average popularity:")
df_genre_counts.show(truncate=False)


Top Performing Genres (Culthit & Superhit Films):
+-------------+--------------+--------------+
|genre_type   |top_film_count|avg_popularity|
+-------------+--------------+--------------+
|multi_genre  |238           |83.193        |
|single_genre |22            |81.039        |
|unknown_genre|1             |68.146        |
+-------------+--------------+--------------+

Number of films by genre type and their average popularity:
+-------------+----------+--------------+
|genre_type   |film_count|avg_popularity|
+-------------+----------+--------------+
|multi_genre  |35991     |3.986         |
|unknown_genre|20839     |0.452         |
|single_genre |48075     |1.215         |
+-------------+----------+--------------+



In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, year

# Create or get a Spark session
spark = SparkSession.builder \
    .appName("DataFrame Analysis") \
    .getOrCreate()

# Load the data from DBFS (ensure df_gold is already loaded)
df_gold = spark.read.parquet("dbfs:/Users/simhad76@students.rowan.edu/silver_layer_data")

# Display original schema
print("\nOriginal Schema:")
df_gold.printSchema()

# Show a sample of data
print("\nSample Data:")
df_gold.show(5, truncate=False)

# Add release_year column using the year function directly on the release_date column
df_gold = df_gold.withColumn("release_year", year(col("release_date")))

# Group by year and count movies
year_counts = df_gold.groupBy("release_year") \
    .agg(count("*").alias("total_movie_count")) \
    .orderBy(col("release_year"))

# Show years from 2000 to 2010 with total movie counts
print("\nMovie counts for years 2000 to 2010:")
year_counts_filtered = year_counts.filter((col("release_year") >= 2000) & (col("release_year") <= 2010))

# Display only total movie counts without average
result_df = year_counts_filtered.select("release_year", "total_movie_count")

result_df.show(truncate=False)






Original Schema:
root
 |-- id: integer (nullable = true)
 |-- popularity: double (nullable = true)
 |-- vote_count: double (nullable = true)
 |-- title: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- Performance_Analysis: string (nullable = true)


Sample Data:
+------+----------+----------+------------------------------+------------+------------+------------------------+-----------------+--------------------+
|id    |popularity|vote_count|title                         |release_date|vote_average|genre                   |original_language|Performance_Analysis|
+------+----------+----------+------------------------------+------------+------------+------------------------+-----------------+--------------------+
|49948 |14.509    |1264.0    |Fantasia 2000                 |2000-01-01  |6.966       |Animation, Family, Music|en            

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, year, month, sum, avg

# Create or get a Spark session
spark = SparkSession.builder \
    .appName("DataFrame Analysis") \
    .getOrCreate()

# Load the data from DBFS (ensure df_gold is already loaded)
df_gold = spark.read.parquet("dbfs:/Users/simhad76@students.rowan.edu/silver_layer_data")

# Display original schema
print("\nOriginal Schema:")
df_gold.printSchema()

# Show a sample of data
print("\nSample Data:")
df_gold.show(5, truncate=False)

# Extract release_year and release_month from the release_date
df_gold = df_gold.withColumn("release_year", year(col("release_date")))
df_gold = df_gold.withColumn("release_month", month(col("release_date")))

# Group by year and month and count movies
year_month_counts = df_gold.groupBy("release_year", "release_month") \
    .agg(count("*").alias("movie_count")) \
    .orderBy(col("release_year"), col("release_month"))

# Calculate total and average movies released per month for years 2000 to 2010
monthly_stats = year_month_counts \
    .filter((col("release_year") >= 2000) & (col("release_year") <= 2010)) \
    .groupBy("release_month") \
    .agg(
        sum("movie_count").alias("total_movie_count"),  # Total number of movies released in this month across years
        avg("movie_count").alias("average_movie_count")  # Average movie count for this month across years
    ) \
    .orderBy("release_month")

# Show total and average movie counts for each month from years 2000 to 2010
print("\nTotal and average movie counts for each month from years 2000 to 2010:")
monthly_stats.show(truncate=False)



Original Schema:
root
 |-- id: integer (nullable = true)
 |-- popularity: double (nullable = true)
 |-- vote_count: double (nullable = true)
 |-- title: string (nullable = true)
 |-- release_date: date (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- Performance_Analysis: string (nullable = true)


Sample Data:
+------+----------+----------+------------------------------+------------+------------+------------------------+-----------------+--------------------+
|id    |popularity|vote_count|title                         |release_date|vote_average|genre                   |original_language|Performance_Analysis|
+------+----------+----------+------------------------------+------------+------------+------------------------+-----------------+--------------------+
|49948 |14.509    |1264.0    |Fantasia 2000                 |2000-01-01  |6.966       |Animation, Family, Music|en            

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, year, desc, first, max, last, min

# Create or get a Spark session
spark = SparkSession.builder \
    .appName("DataFrame Analysis") \
    .getOrCreate()

# Load the data from DBFS (ensure df_gold is already loaded)
df_gold = spark.read.parquet("dbfs:/Users/simhad76@students.rowan.edu/silver_layer_data")

# Check for null values in release_date
null_release_date_count = df_gold.filter(col("release_date").isNull()).count()
print(f"Number of null rows in release_date column: {null_release_date_count}")

# Extract year directly from release_date since it is already in date format
df_gold = df_gold.withColumn("release_year", year(col("release_date")))

# Filter out rows with null release_year before aggregation
df_gold_filtered = df_gold.filter(col("release_year").isNotNull())

# Group by release_year and original_language and count movies
language_year_counts = df_gold_filtered.groupBy("release_year", "original_language") \
    .agg(count("*").alias("movie_count")) \
    .orderBy("release_year", desc("movie_count"))

# Get the most and least movies released in each language for each year
most_least_movies_per_year = language_year_counts \
    .groupBy("release_year") \
    .agg(
        first("original_language").alias("most_frequent_language"),
        max("movie_count").alias("max_movie_count"),
        last("original_language").alias("least_frequent_language"),  # Get last entry for least frequent
        min("movie_count").alias("min_movie_count")
    )

# Show results for each year with the most and least frequent languages and their movie counts
print("Most and least movies released in each language per year:")
most_least_movies_per_year.show(truncate=False)


Number of null rows in release_date column: 2
Most and least movies released in each language per year:
+------------+----------------------+---------------+-----------------------+---------------+
|release_year|most_frequent_language|max_movie_count|least_frequent_language|min_movie_count|
+------------+----------------------+---------------+-----------------------+---------------+
|2000        |en                    |3866           |am                     |1              |
|2001        |en                    |4203           |wo                     |1              |
|2002        |en                    |4736           |iu                     |1              |
|2003        |en                    |5222           |ht                     |1              |
|2004        |en                    |5039           |iu                     |1              |
|2005        |en                    |4993           |ht                     |1              |
|2006        |en                    |5203         

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg, count, desc

# Initialize Spark session
spark = SparkSession.builder.appName("Movie Vote Analysis").getOrCreate()

# Assuming df_gold is already defined and contains your data
# Group by original_language and calculate total and average vote counts
language_vote_counts = df_gold.groupBy("original_language") \
    .agg(
        sum("vote_count").alias("total_vote_count"),
        avg("vote_count").alias("average_vote_count"),
        count("*").alias("movie_count")  # Count of movies for calculating average votes
    ) \
    .orderBy(desc("total_vote_count"))

# Show all languages with their total and average vote counts
print("Total and average vote count for all languages:")
language_vote_counts.show(truncate=False)

# Show the language with the highest total vote count at the top
top_language = language_vote_counts.first()  # Get the top voted language
print("\nLanguage with the highest vote count:")
print(f"Language: {top_language['original_language']}, Total Vote Count: {top_language['total_vote_count']}, Average Vote Count: {top_language['average_vote_count']:.2f}, Total Movies Released: {top_language['movie_count']}")

# Calculate total number of languages
total_languages = language_vote_counts.count()  # Use language_vote_counts instead of language_counts
print(f"\nTotal number of languages: {total_languages}")




Total and average vote count for all languages:
+-----------------+----------------+------------------+-----------+
|original_language|total_vote_count|average_vote_count|movie_count|
+-----------------+----------------+------------------+-----------+
|en               |5605743.0       |104.9410872739526 |53418      |
|fr               |196931.0        |30.574600217357553|6441       |
|ja               |131073.0        |21.12717601547389 |6204       |
|es               |83917.0         |14.473439116936875|5798       |
|it               |68081.0         |30.285142348754448|2248       |
|ko               |57599.0         |46.11609287429944 |1249       |
|de               |48221.0         |8.476182105818246 |5689       |
|cn               |32397.0         |36.11705685618729 |897        |
|hi               |31158.0         |26.585324232081913|1172       |
|zh               |27969.0         |14.248089658685686|1963       |
|pt               |27836.0         |11.09003984063745 |2510       |


In [0]:

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, count, avg, sum as _sum, when, split, size, round

# Create or get a Spark session
spark = SparkSession.builder.appName("Movie Data Processing").getOrCreate()

# Load the silver layer data
df_silver = spark.read.parquet("dbfs:/Users/simhad76@students.rowan.edu/silver_layer_data")

def process_movie_data(df: DataFrame) -> DataFrame:
    """
    Process the movie DataFrame to classify genres and perform aggregations.
    
    Args:
        df (DataFrame): Input DataFrame containing movie data.
        
    Returns:
        DataFrame: Final processed DataFrame with aggregated metrics and genre classification.
    """
    # Check for null values in the genre column
    null_genre_count = df.filter(col("genre").isNull()).count()
    print(f"Number of null rows in genre column: {null_genre_count}")

    # Count distinct genres
    distinct_genres = df.select("genre").distinct().count()
    print(f"Number of distinct genres: {distinct_genres}")

    # Determine if a film is multi-genre, single-genre, or unknown genre
    df = df.withColumn(
        "genre_count",
        size(split(col("genre"), ","))
    ).withColumn(
        "genre_type",
        when((col("genre") == "Unknown") | (col("genre").startswith("Unknown")) & (col("genre_count") == 1), "unknown_genre")
        .when(col("genre_count") > 1, "multi_genre")
        .otherwise("single_genre")
    )

    # Aggregate metrics to create the gold layer based on original_language
    gold_layer = df.groupBy("original_language").agg(
        _sum("vote_count").alias("total_vote_count"),
        avg("vote_count").alias("average_vote_count"),
        count("*").alias("movie_count")
    ).orderBy(col("total_vote_count").desc())

    # Merge gold layer with the original DataFrame based on original_language
    merged_df = df.join(gold_layer, on="original_language", how="inner")

    # Select relevant columns from merged DataFrame including new columns only
    final_df = merged_df.select(
        "original_language",
        "title",  # Title from silver layer
        "genre",  # Genre from silver layer
        "release_date",  # Release date from silver layer
        "total_vote_count",  # Total vote count from gold layer
        "average_vote_count",  # Average vote count from gold layer
        "movie_count",  # Movie count from gold layer
        "genre_type",  # Add genre_type to final output
        "Performance_Analysis",
    )

    return final_df

# Process the movie data
final_df_loaded = process_movie_data(df_silver)

# Show the final merged DataFrame including genre_type
final_df_loaded.show(truncate=False)

# Write the merged DataFrame to persistent storage
final_df_loaded.write.format("delta").mode("overwrite").save("dbfs:/Users/simhad76@students.rowan.edu/moviedata")

# Load the final merged DataFrame from Delta table for inspection (if needed)
final_df = spark.read.format("delta").load("dbfs:/Users/simhad76@students.rowan.edu/moviedata")

# Show the loaded data from Delta table
print("Data loaded from Delta table:")
final_df.show(truncate=False)


Number of null rows in genre column: 0
Number of distinct genres: 4225
+-----------------+--------------------------------+------------------------+------------+----------------+------------------+-----------+-------------+--------------------+
|original_language|title                           |genre                   |release_date|total_vote_count|average_vote_count|movie_count|genre_type   |Performance_Analysis|
+-----------------+--------------------------------+------------------------+------------+----------------+------------------+-----------+-------------+--------------------+
|en               |Fantasia 2000                   |Animation, Family, Music|2000-01-01  |5605743.0       |104.9410872739526 |53418      |multi_genre  |Hit                 |
|ko               |Peppermint Candy                |Drama                   |2000-01-01  |57599.0         |46.11609287429944 |1249       |single_genre |Hit                 |
|en               |Nautilus                        |Action,

In [0]:

from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.functions import col, count, avg, sum as _sum, when, split, size, round, expr

def process_movie_data(df: DataFrame) -> DataFrame:
    """
    Process the movie DataFrame to classify genres and perform aggregations,
    and calculate year-over-year growth for movies by language and year.
    
    Args:
        df (DataFrame): Input DataFrame containing movie data.
        
    Returns:
        DataFrame: Final processed DataFrame with aggregated metrics, genre classification,
                    average popularity, and year-over-year growth.
    """
    # Check for null values in the genre column
    null_genre_count = df.filter(col("genre").isNull()).count()
    print(f"Number of null rows in genre column: {null_genre_count}")

    # Count distinct genres
    distinct_genres = df.select("genre").distinct().count()
    print(f"Number of distinct genres: {distinct_genres}")

    # Determine if a film is multi-genre, single-genre, or unknown genre
    df = df.withColumn(
        "genre_count",
        size(split(col("genre"), ","))
    ).withColumn(
        "genre_type",
        when(col("genre").startswith("Unknown"), "unknown_genre")
        .when(col("genre_count") > 1, "multi_genre")
        .otherwise("single_genre")
    )

    # Calculate average popularity for each genre type
    avg_popularity_by_genre = df.groupBy("genre_type").agg(
        round(avg("popularity"), 3).alias("average_popularity")  # Round to 3 decimals
    )

    # Aggregate metrics to create the gold layer based on original_language
    gold_layer = df.groupBy("original_language").agg(
        _sum("vote_count").alias("total_vote_count"),
        avg("vote_count").alias("average_vote_count"),
        count("*").alias("movie_count")
    ).orderBy(col("total_vote_count").desc())

    # Merge gold layer with the original DataFrame based on original_language
    merged_df = df.join(gold_layer, on="original_language", how="inner")

    # Join with average popularity by genre type
    merged_df = merged_df.join(avg_popularity_by_genre, on="genre_type", how="left")

    # Convert release_date to date type and extract year for YOY growth analysis
    df = df.withColumn("release_year", expr("year(to_date(release_date, 'M/d/yyyy'))"))

    # Filter out rows with null release_year before aggregation
    df_filtered = df.filter(col("release_year").isNotNull())

    # Group by release_year and original_language and count movies
    language_year_counts = df_filtered.groupBy("release_year", "original_language") \
        .agg(count("*").alias("movie_count")) \
        .orderBy("release_year")

    # Calculate YOY growth
    yoy_growth_df = (
        language_year_counts
        .groupBy("original_language")
        .pivot("release_year")  # Automatically pivots years based on available data
        .agg(_sum("movie_count").alias("movie_count"))
        .fillna(0)  # Fill missing values with 0 for years with no movies
    )

    # Add yoy_growth column based on movie counts for specific years (adjust years as needed)
    yoy_growth_df = yoy_growth_df.withColumn(
        "yoy_growth",
        when(col("2009") < col("2010"), "High Growth")
        .when(col("2009") == col("2010"), "No Growth")
        .otherwise("Low Growth")
    )

    print("\nYear-Over-Year Growth by Original Language:")
    yoy_growth_df.show(truncate=False)

    # Join YOY growth back to the final DataFrame based on original_language
    final_df = merged_df.join(yoy_growth_df.select("original_language", "yoy_growth"), 
                               on="original_language", 
                               how="left")

    # Drop specified columns from the final DataFrame
    final_df = final_df.drop(
        "previous_year_movies",
        "current_year_movies",
        "vote_count",
        "vote_average"
    )

    return final_df

# Example usage:
# Assuming df_silver is your initial DataFrame loaded from DBFS.
final_df_loaded = process_movie_data(df_silver)

# Show the final merged DataFrame including genre_type and average popularity with YOY growth
print("\nFinal Merged DataFrame with Genre Type, Average Popularity, and YOY Growth:")
final_df_loaded.show(truncate=False)

# Optionally write the merged DataFrame to persistent storage for further use.
final_df_loaded.write.format("delta").mode("overwrite").save("dbfs:/Users/simhad76@students.rowan.edu/moviedata")

# Load the final merged DataFrame from Delta table for inspection (if needed)
final_df = spark.read.format("delta").load("dbfs:/Users/simhad76@students.rowan.edu/moviedata")

# Show number of columns and their schema of the loaded DataFrame
num_columns = len(final_df.columns)
print(f"\nNumber of columns in the merged DataFrame: {num_columns}")

# Show number of rows in the loaded DataFrame
num_rows = final_df.count()
print(f"\nNumber of rows in the merged DataFrame: {num_rows}")

# Print schema of the loaded DataFrame
print("\nSchema of the merged DataFrame:")
final_df.printSchema()


Number of null rows in genre column: 0
Number of distinct genres: 4225

Year-Over-Year Growth by Original Language:
+-----------------+----+----+----+----+----+----+----+----+----+----+----+-----------+
|original_language|2000|2001|2002|2003|2004|2005|2006|2007|2008|2009|2010|yoy_growth |
+-----------------+----+----+----+----+----+----+----+----+----+----+----+-----------+
|en               |3866|4203|4736|5222|5039|4993|5203|5082|5035|5090|4949|Low Growth |
|cr               |1   |0   |0   |0   |1   |0   |1   |0   |0   |1   |0   |Low Growth |
|fy               |1   |1   |0   |0   |0   |0   |0   |0   |0   |0   |0   |No Growth  |
|vi               |2   |5   |6   |9   |8   |11  |8   |4   |5   |9   |6   |Low Growth |
|ne               |2   |2   |2   |3   |2   |2   |1   |3   |2   |3   |4   |High Growth|
|nb               |0   |1   |1   |0   |2   |2   |1   |2   |0   |3   |1   |Low Growth |
|ps               |0   |0   |0   |1   |2   |0   |0   |1   |0   |0   |0   |No Growth  |
|ro           

In [0]:
# Save final_updated_loaded as a table in Spark SQL (Hive metastore)
table_name = "tmbdApi_movies_data"  # Specify your desired table name

final_df.write.format("delta").mode("overwrite").saveAsTable(table_name)

print(f"\nTable '{table_name}' has been created successfully.")


Table 'tmbdApi_movies_data' has been created successfully.


In [0]:
# Load the Delta table into a DataFrame
table_name = "tmbdApi_movies_data"  # Use the name of your table
delta_table_df = spark.read.format("delta").table(table_name)

# Show the contents of the Delta table
print(f"\nContents of the table '{table_name}':")
delta_table_df.show(truncate=False)  # Use truncate=False to see full column values


Contents of the table 'tmbdApi_movies_data':
+-----------------+--------------------------------+------------------------+------------+----------------+------------------+-----------+-------------+--------------------+------+----------+-----------+------------------+-----------+
|original_language|title                           |genre                   |release_date|total_vote_count|average_vote_count|movie_count|genre_type   |Performance_Analysis|id    |popularity|genre_count|average_popularity|yoy_growth |
+-----------------+--------------------------------+------------------------+------------+----------------+------------------+-----------+-------------+--------------------+------+----------+-----------+------------------+-----------+
|en               |Fantasia 2000                   |Animation, Family, Music|2000-01-01  |5605743.0       |104.9410872739526 |53418      |multi_genre  |Hit                 |49948 |14.509    |3          |3.986             |Low Growth |
|ko           